In [1]:
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import os

import glob

files = glob.glob('/home/freinque/cours/voice/sillytalks/data/pickles/train/*.p')
print len(files)
files = glob.glob('/home/freinque/cours/voice/sillytalks/data/pickles/pipeline_train/*.p')
print len(files)

files = glob.glob('/home/freinque/cours/voice/sillytalks/data/pickles/test/*.p')
print len(files)
files = glob.glob('/home/freinque/cours/voice/sillytalks/data/pickles/pipeline_test/*.p')
print len(files)

64727
64727
158538
8898


<br><br><br><br><br><br><br><br><br><br>
# generate train

In [3]:
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import os

import glob

files = glob.glob('/home/freinque/cours/voice/sillytalks/data/pickles/pipeline_train/*.p')

Y_train = []

for f in files[:1000]:

        target = os.path.basename(f).split('_')[0]
    
        Y_train.append(target)

Y_train = np.array(Y_train)
labels = list(np.sort(np.unique(Y_train)))

pickle.dump( labels, open( '/home/freinque/cours/voice/sillytalks/data/labels.p', "wt" ) )

In [1]:
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import os

import glob

from keras.utils import np_utils

def batch_generator(files, labels, batch_size, num_classes):


    while True:
        #print 'generator called: '
        X_train = []
        Y_train = []
    
        fil = np.random.choice(files, batch_size)
        #print fil
        for f in fil:

            target = os.path.basename(f).split('_')[0]
            feat = pickle.load( open(f, 'rb'))
    
            X_train.append( np.array([ np.array(feat) ]) )
            Y_train.append(target)
            
        X_train = np.array(X_train)
        #print 'X_train.shape', X_train.shape
        
        Y_train = np.array(Y_train)
        Y_train = np.array( [ labels.index(y) for y in Y_train ] )
        Y_train = np_utils.to_categorical(Y_train, num_classes)
        #print 'Y_train.shpe', Y_train.shape
        
        yield X_train, Y_train
    

Using TensorFlow backend.


# keras train

In [2]:
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.models import Model
from keras.layers import Input

num_classes = 30

    # layers
conv_55_1 = Conv2D(32, (5, 5), padding='same', activation='relu', kernel_constraint=maxnorm(3), data_format="channels_first")
dropout_20 = Dropout(0.2)
conv_55_2 = Conv2D(32, (5, 5), activation='relu', padding='same', kernel_constraint=maxnorm(3), data_format="channels_first")
max_pooling = MaxPooling2D(pool_size=(5, 5))

flatten = Flatten()

dense_128_1 = Dense(128, activation='relu', kernel_constraint=maxnorm(3))
dense_128_2 = Dense(128, activation='relu', kernel_constraint=maxnorm(3))
dropout_50 = Dropout(0.5)
final_layer = Dense(num_classes, activation='softmax')

    # model
inp = Input(shape=(1,41, 615))
out = final_layer(dropout_50(dense_128_2(dense_128_1(flatten(max_pooling(conv_55_2(dropout_20(conv_55_1(inp)))))))))

model = Model(inputs=inp, outputs=out)

    # Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs

sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

model_name = 'base_1'
model_path = '/home/freinque/cours/voice/sillytalks/data/models'
model.load_weights(os.path.join(model_path, model_name+'.hdf5'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 41, 615)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 41, 615)       832       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 41, 615)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 41, 615)       25632     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 8, 615)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 29520)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               3778688   
__________

In [6]:
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint

model_name = 'base_1'
model_path = '/home/freinque/cours/voice/sillytalks/data/models'
checkpointer = ModelCheckpoint(filepath=os.path.join(model_path, model_name+'.hdf5'), verbose=1, save_best_only=False)

files = glob.glob('/home/freinque/cours/voice/sillytalks/data/pickles/pipeline_train/*.p')
labels = pickle.load(open('/home/freinque/cours/voice/sillytalks/data/labels.p', 'rb'))
batch_size = 32

model.fit_generator(
    batch_generator(files, labels, batch_size, num_classes), 
    validation_data = batch_generator(files, labels, batch_size, num_classes),
    validation_steps = 10,
    steps_per_epoch=50, 
    epochs=10,
    workers=4, use_multiprocessing=True,
    callbacks=[checkpointer,],
)

Epoch 1/10
50/50 [==============================] - 2068s 41s/step - loss: 3.3889 - acc: 0.0438 - val_loss: 3.3524 - val_acc: 0.0469
Epoch 2/10
50/50 [==============================] - 2045s 41s/step - loss: 3.2633 - acc: 0.0887 - val_loss: 3.0109 - val_acc: 0.1625
Epoch 3/10
50/50 [==============================] - 3246s 65s/step - loss: 3.0578 - acc: 0.1281 - val_loss: 2.8668 - val_acc: 0.1562
Epoch 4/10
50/50 [==============================] - 3236s 65s/step - loss: 2.8262 - acc: 0.1800 - val_loss: 2.6967 - val_acc: 0.2313
Epoch 5/10
50/50 [==============================] - 3227s 65s/step - loss: 2.7276 - acc: 0.2194 - val_loss: 2.3918 - val_acc: 0.2844
Epoch 6/10
50/50 [==============================] - 2131s 43s/step - loss: 2.5870 - acc: 0.2525 - val_loss: 2.3197 - val_acc: 0.3125
Epoch 7/10
50/50 [==============================] - 1528s 31s/step - loss: 2.5135 - acc: 0.2781 - val_loss: 2.2919 - val_acc: 0.3312
Epoch 8/10
50/50 [==============================] - 1408s 28s/step - 

Traceback (most recent call last):
  File "/home/freinque/anaconda2/lib/python2.7/site-packages/keras/utils/data_utils.py", line 635, in data_generator_task
    generator_output = next(self._generator)
  File "<ipython-input-1-44a19de801f6>", line 33, in batch_generator
    Y_train = np.array( [ labels.index(y) for y in Y_train ] )
ValueError: '' is not in list


34/50 [===================>..........] - ETA: 16:07 - loss: 2.2486 - acc: 0.3355

ValueError: '' is not in list

In [9]:
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint

model_name = 'base_1'
model_path = '/home/freinque/cours/voice/sillytalks/data/models'
checkpointer = ModelCheckpoint(filepath=os.path.join(model_path, model_name+'.hdf5'), verbose=1, save_best_only=False)

files = glob.glob('/home/freinque/cours/voice/sillytalks/data/pickles/pipeline_train/*.p')
labels = pickle.load(open('/home/freinque/cours/voice/sillytalks/data/labels.p', 'rb'))
batch_size = 32

model.fit_generator(
    batch_generator(files, labels, batch_size, num_classes), 
    validation_data = batch_generator(files, labels, batch_size, num_classes),
    validation_steps = 10,
    steps_per_epoch=50, 
    epochs=10,
    workers=4, use_multiprocessing=True,
    callbacks=[checkpointer,],
)

Epoch 1/10
50/50 [==============================] - 2348s 47s/step - loss: 2.1416 - acc: 0.3725 - val_loss: 2.1273 - val_acc: 0.4406
Epoch 2/10
50/50 [==============================] - 2276s 46s/step - loss: 2.1910 - acc: 0.3581 - val_loss: 2.0377 - val_acc: 0.3906
Epoch 3/10
50/50 [==============================] - 2181s 44s/step - loss: 2.0230 - acc: 0.4012 - val_loss: 1.8998 - val_acc: 0.4594
Epoch 4/10


Traceback (most recent call last):
  File "/home/freinque/anaconda2/lib/python2.7/site-packages/keras/utils/data_utils.py", line 635, in data_generator_task
    generator_output = next(self._generator)
  File "<ipython-input-1-44a19de801f6>", line 29, in batch_generator
    X_train = np.array(X_train)
ValueError: could not broadcast input array from shape (41,615) into shape (1)


 5/50 [==>...........................] - ETA: 34:03 - loss: 2.1609 - acc: 0.3812

ValueError: could not broadcast input array from shape (41,615) into shape (1)

In [10]:
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint

model_name = 'base_1'
model_path = '/home/freinque/cours/voice/sillytalks/data/models'
checkpointer = ModelCheckpoint(filepath=os.path.join(model_path, model_name+'.hdf5'), verbose=1, save_best_only=False)

files = glob.glob('/home/freinque/cours/voice/sillytalks/data/pickles/pipeline_train/*.p')
labels = pickle.load(open('/home/freinque/cours/voice/sillytalks/data/labels.p', 'rb'))
batch_size = 32

model.fit_generator(
    batch_generator(files, labels, batch_size, num_classes), 
    validation_data = batch_generator(files, labels, batch_size, num_classes),
    validation_steps = 10,
    steps_per_epoch=50, 
    epochs=10,
    workers=4, use_multiprocessing=True,
    callbacks=[checkpointer,],
)

Epoch 1/10
50/50 [==============================] - 2403s 48s/step - loss: 2.0543 - acc: 0.3925 - val_loss: 1.6597 - val_acc: 0.5563
Epoch 2/10
50/50 [==============================] - 2341s 47s/step - loss: 1.9209 - acc: 0.4144 - val_loss: 1.7342 - val_acc: 0.5250
Epoch 3/10
50/50 [==============================] - 2502s 50s/step - loss: 1.9310 - acc: 0.4263 - val_loss: 1.6783 - val_acc: 0.4750
Epoch 4/10
50/50 [==============================] - 2519s 50s/step - loss: 1.8744 - acc: 0.4419 - val_loss: 1.6294 - val_acc: 0.5125
Epoch 5/10
50/50 [==============================] - 2492s 50s/step - loss: 1.8069 - acc: 0.4656 - val_loss: 1.6649 - val_acc: 0.5531
Epoch 6/10
50/50 [==============================] - 2444s 49s/step - loss: 1.7620 - acc: 0.4713 - val_loss: 1.5576 - val_acc: 0.5500
Epoch 7/10
50/50 [==============================] - 7725s 155s/step - loss: 1.7723 - acc: 0.4706 - val_loss: 1.5125 - val_acc: 0.5531
Epoch 8/10
 1/50 [..............................] - ETA: 9:54 - loss

KeyboardInterrupt: 

In [11]:
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint

model_name = 'base_1'
model_path = '/home/freinque/cours/voice/sillytalks/data/models'
checkpointer = ModelCheckpoint(filepath=os.path.join(model_path, model_name+'.hdf5'), verbose=1, save_best_only=False)

files = glob.glob('/home/freinque/cours/voice/sillytalks/data/pickles/pipeline_train/*.p')
labels = pickle.load(open('/home/freinque/cours/voice/sillytalks/data/labels.p', 'rb'))
batch_size = 32

model.fit_generator(
    batch_generator(files, labels, batch_size, num_classes), 
    validation_data = batch_generator(files, labels, batch_size, num_classes),
    validation_steps = 10,
    steps_per_epoch=50, 
    epochs=10,
    workers=4, use_multiprocessing=True,
    callbacks=[checkpointer,],
)

Epoch 1/10
50/50 [==============================] - 764s 15s/step - loss: 1.6934 - acc: 0.4862 - val_loss: 1.5176 - val_acc: 0.5844
Epoch 2/10
50/50 [==============================] - 667s 13s/step - loss: 1.6721 - acc: 0.5112 - val_loss: 1.3594 - val_acc: 0.5813
Epoch 3/10
50/50 [==============================] - 658s 13s/step - loss: 1.6133 - acc: 0.5188 - val_loss: 1.4513 - val_acc: 0.6000
Epoch 4/10
50/50 [==============================] - 657s 13s/step - loss: 1.5727 - acc: 0.5306 - val_loss: 1.4352 - val_acc: 0.5563
Epoch 5/10
50/50 [==============================] - 667s 13s/step - loss: 1.5431 - acc: 0.5419 - val_loss: 1.3079 - val_acc: 0.6250
Epoch 6/10
50/50 [==============================] - 667s 13s/step - loss: 1.5456 - acc: 0.5394 - val_loss: 1.4147 - val_acc: 0.5719
Epoch 7/10
50/50 [==============================] - 666s 13s/step - loss: 1.5369 - acc: 0.5238 - val_loss: 1.3173 - val_acc: 0.6531
Epoch 8/10
50/50 [==============================] - 657s 13s/step - loss: 1.

In [12]:
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint

model_name = 'base_1'
model_path = '/home/freinque/cours/voice/sillytalks/data/models'
checkpointer = ModelCheckpoint(filepath=os.path.join(model_path, model_name+'.hdf5'), verbose=1, save_best_only=False)

files = glob.glob('/home/freinque/cours/voice/sillytalks/data/pickles/pipeline_train/*.p')
labels = pickle.load(open('/home/freinque/cours/voice/sillytalks/data/labels.p', 'rb'))
batch_size = 32

model.fit_generator(
    batch_generator(files, labels, batch_size, num_classes), 
    validation_data = batch_generator(files, labels, batch_size, num_classes),
    validation_steps = 10,
    steps_per_epoch=50, 
    epochs=10,
    workers=4, use_multiprocessing=True,
    callbacks=[checkpointer,],
)

Epoch 1/10
50/50 [==============================] - 665s 13s/step - loss: 1.4151 - acc: 0.5831 - val_loss: 1.2898 - val_acc: 0.6562
Epoch 2/10
50/50 [==============================] - 659s 13s/step - loss: 1.4438 - acc: 0.5637 - val_loss: 1.2952 - val_acc: 0.6469
Epoch 3/10
50/50 [==============================] - 659s 13s/step - loss: 1.4072 - acc: 0.5844 - val_loss: 1.1547 - val_acc: 0.6719
Epoch 4/10
50/50 [==============================] - 656s 13s/step - loss: 1.3561 - acc: 0.5963 - val_loss: 1.1366 - val_acc: 0.6937
Epoch 5/10
50/50 [==============================] - 659s 13s/step - loss: 1.3472 - acc: 0.6025 - val_loss: 1.0130 - val_acc: 0.6969
Epoch 6/10
50/50 [==============================] - 669s 13s/step - loss: 1.4352 - acc: 0.5600 - val_loss: 1.2122 - val_acc: 0.6438
Epoch 7/10
50/50 [==============================] - 659s 13s/step - loss: 1.2608 - acc: 0.6225 - val_loss: 0.9749 - val_acc: 0.7063
Epoch 8/10
50/50 [==============================] - 659s 13s/step - loss: 1.

In [13]:
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint

model_name = 'base_1'
model_path = '/home/freinque/cours/voice/sillytalks/data/models'
checkpointer = ModelCheckpoint(filepath=os.path.join(model_path, model_name+'.hdf5'), verbose=1, save_best_only=False)

files = glob.glob('/home/freinque/cours/voice/sillytalks/data/pickles/pipeline_train/*.p')
labels = pickle.load(open('/home/freinque/cours/voice/sillytalks/data/labels.p', 'rb'))
batch_size = 32

model.fit_generator(
    batch_generator(files, labels, batch_size, num_classes), 
    validation_data = batch_generator(files, labels, batch_size, num_classes),
    validation_steps = 10,
    steps_per_epoch=50, 
    epochs=10,
    workers=4, use_multiprocessing=True,
    callbacks=[checkpointer,],
)

Epoch 1/10
50/50 [==============================] - 864s 17s/step - loss: 1.2438 - acc: 0.6206 - val_loss: 1.0429 - val_acc: 0.7188
Epoch 2/10
50/50 [==============================] - 663s 13s/step - loss: 1.2586 - acc: 0.6162 - val_loss: 1.0863 - val_acc: 0.6906
Epoch 3/10
50/50 [==============================] - 665s 13s/step - loss: 1.2421 - acc: 0.6281 - val_loss: 0.8814 - val_acc: 0.7625
Epoch 4/10
50/50 [==============================] - 662s 13s/step - loss: 1.2148 - acc: 0.6300 - val_loss: 1.1284 - val_acc: 0.6813
Epoch 5/10
50/50 [==============================] - 661s 13s/step - loss: 1.2099 - acc: 0.6338 - val_loss: 1.0165 - val_acc: 0.7063
Epoch 6/10
50/50 [==============================] - 665s 13s/step - loss: 1.1299 - acc: 0.6481 - val_loss: 0.9677 - val_acc: 0.7063
Epoch 7/10
50/50 [==============================] - 661s 13s/step - loss: 1.2015 - acc: 0.6269 - val_loss: 0.8934 - val_acc: 0.7500
Epoch 8/10
50/50 [==============================] - 662s 13s/step - loss: 1.

In [ ]:
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint

class save_weights(Callback):

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        


model.fit(
          X_train[:int(0.8*len(X_train))], Y_train[:int(0.8*len(X_train))], 
          validation_data=(X_train[int(0.8*len(X_train)):], Y_train[int(0.8*len(X_train)):]), 
          epochs=epochs, 
          batch_size=32,
          callbacks=[checkpointer,],
         )

In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_train[0.8*len(X_train):], Y_train[0.8*len(X_train):], verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
X_train.shape

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [ ]:
y_train

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))